In [ ]:
pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()

In [2]:
data1 = [("Alice", 1, 1000, "Address1"),
         ("Bob", 2, 2000, "Address2"),
         ("Charlie", 3, 3000, "Address3"),
         ("David", 4, 4000, "Address4"),
         ("Eve", 5, 5000, "Address5")]

data2 = [(1, 25, "Married"),
         (2, 30, "Single"),
         (3, 28, "Married"),
         (4, 32, "Single"),
         (5, 27, "Married")]

In [7]:
df1 = spark.createDataFrame(data1, ["student_name", "id", "salary", "address"])
df2 = spark.createDataFrame(data2, ["id", "age", "married_status"])


In [8]:
joined_df=df1.join(df2,"id")
joined_df.show()

+---+------------+------+--------+---+--------------+
| id|student_name|salary| address|age|married_status|
+---+------------+------+--------+---+--------------+
|  1|       Alice|  1000|Address1| 25|       Married|
|  2|         Bob|  2000|Address2| 30|        Single|
|  3|     Charlie|  3000|Address3| 28|       Married|
|  4|       David|  4000|Address4| 32|        Single|
|  5|         Eve|  5000|Address5| 27|       Married|
+---+------------+------+--------+---+--------------+



# <mark>_<u>**Broad Case Join**</u>_</mark>

In [9]:
data3 = [(1, "DeptA"),
         (2, "DeptB"),
         (3, "DeptC"),
         (4, "DeptD"),
         (5, "DeptE")]

df3=spark.createDataFrame(data3, ["id","department"])

joined_df = df1.join(df2, "id").join(df3.hint("broadcast"), "id")

joined_df.show()

+---+------------+------+--------+---+--------------+----------+
| id|student_name|salary| address|age|married_status|department|
+---+------------+------+--------+---+--------------+----------+
|  1|       Alice|  1000|Address1| 25|       Married|     DeptA|
|  2|         Bob|  2000|Address2| 30|        Single|     DeptB|
|  3|     Charlie|  3000|Address3| 28|       Married|     DeptC|
|  4|       David|  4000|Address4| 32|        Single|     DeptD|
|  5|         Eve|  5000|Address5| 27|       Married|     DeptE|
+---+------------+------+--------+---+--------------+----------+

